In [13]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import os
import pandas as pd

import datetime
import time
from yahoo_historical import Fetcher



os.chdir(r'C:\Users\hadit\OneDrive - Prophix Software, Inc\AutoTrading')
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg
 
# load dataset
ticker = 'ETH-CAD'
#dstart = '1-1-2021' #start date of data to calculate rolling averages
actual_start = '1-1-2022' # actual start date where data is analyzed

dstart = datetime.datetime.strptime(actual_start, '%m-%d-%Y')
print(dstart)
dstart = time.mktime(dstart.timetuple())
df = Fetcher(ticker, dstart).get_historical()
df.Date = pd.to_datetime(df.Date)
dates = df.Date
df.drop(['Date','Adj Close'], axis=1, inplace=True)
df.columns

y = df.Close - df.Close.shift(1)
y = y>0
y  = y.astype(int)
df['y'] = y

N = df.shape[1]
values = df.values
# integer encode direction

#values[:,4] = encoder.fit_transform(values[:,4])
# ensure all data is float
values = values.astype('float32')
# normalize features
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(values)
# frame as supervised learning
n_in = 2
reframed = series_to_supervised(values, n_in, 1)
# drop columns we don't want to predict
#reframed.drop(reframed.columns[[9,10,11,12,13,14,15]], axis=1, inplace=True)
reframed.drop(['var'+str(i)+'(t)' for i in range(1,N)], axis=1, inplace=True)
reframed.drop(['var'+str(N)+'(t-' + str(t) + ')' for t in range(1,n_in+1)], axis=1, inplace=True)

print(reframed.columns)
reframed
 

2022-01-01 00:00:00
Index(['var1(t-2)', 'var2(t-2)', 'var3(t-2)', 'var4(t-2)', 'var5(t-2)',
       'var1(t-1)', 'var2(t-1)', 'var3(t-1)', 'var4(t-1)', 'var5(t-1)',
       'var6(t)'],
      dtype='object')


,var1(t-2),var2(t-2),var3(t-2),var4(t-2),var5(t-2),var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t)
2,4655.555664,4765.364746,4654.594238,4765.085449,1.235760e+10,4764.581543,4842.118652,4711.566406,4848.099121,1.250961e+10,0.0
3,4764.581543,4842.118652,4711.566406,4848.099121,1.250961e+10,4848.062012,4888.744141,4717.070801,4796.455566,1.540523e+10,1.0
4,4848.062012,4888.744141,4717.070801,4796.455566,1.540523e+10,4796.431641,4927.568359,4744.676270,4820.447754,1.782666e+10,0.0
5,4796.431641,4927.568359,4744.676270,4820.447754,1.782666e+10,4820.717773,4883.027344,4409.953125,4530.520996,2.304742e+10,0.0
6,4820.717773,4883.027344,4409.953125,4530.520996,2.304742e+10,4529.655762,4529.655762,4264.112305,4351.890137,3.092782e+10,0.0
...,...,...,...,...,...,...,...,...,...,...,...
364,1618.529297,1631.660034,1615.487915,1628.673584,5.600937e+09,1628.638672,1629.492676,1608.004761,1627.291748,5.503315e+09,0.0
365,1628.638672,1629.492676,1608.004761,1627.291748,5.503315e+09,1627.464478,1635.245117,1620.474731,1623.958740,4.095972e+09,1.0
366,1627.464478,1635.245117,1620.474731,1623.958740,4.095972e+09,1623.880615,1631.383057,1618.685913,1625.373779,3.247696e+09,1.0
367,1623.880615,1631.383057,1618.685913,1625.373779,3.247696e+09,1625.561157,1656.265747,1616.793701,1647.529907,5.107781e+09,1.0


In [14]:
# split into train and test sets
values = reframed.values
#n_train_hours = 365 * 24
train = values[:round(0.8*values.shape[0]), :]
test = values[round(0.8*values.shape[0]):, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]


print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(294, 10) (294,) (73, 10) (73,)


In [16]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(train_X, train_y)
y_pred = model.predict(test_X)
acc_base_logreg = accuracy_score(test_y, y_pred)
acc_base_logreg

0.4383561643835616

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV


# Number of trees in random forest
import numpy as np
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
random_grid

rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random.fit(train_X, train_y)
rf_random.best_params_

from sklearn.metrics import accuracy_score
y_pred = rf_random.predict(test_X)

acc_tuned = accuracy_score(y_pred, test_y)

rf = RandomForestClassifier(n_estimators = 100, random_state = 0)
rf.fit(train_X, train_y)
y_pred = rf.predict(test_X)
acc_base = accuracy_score(test_y, y_pred)


from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(train_X, train_y)
y_pred = model.predict(test_X)
acc_xgb = accuracy_score(test_y, y_pred)


Fitting 3 folds for each of 20 candidates, totalling 60 fits


In [18]:
acc_tuned

0.4383561643835616

In [19]:
acc_base

0.4657534246575342

In [20]:
acc_xgb

0.4246575342465753

In [21]:
rf_random.best_params_

{'n_estimators': 600,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 70,
 'bootstrap': True}

In [22]:
from mango import Tuner, scheduler


In [23]:
param_space = dict(n_estimators=n_estimators,
                   max_features = max_features,
                   max_depth = max_depth
                   )
